## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bluff,NZ,-46.6000,168.3333,50.81,85,11,3.51,few clouds
1,1,Faya,SA,18.3851,42.4509,61.32,66,4,1.92,clear sky
2,2,Rabo De Peixe,PT,37.8000,-25.5833,55.27,81,20,11.50,few clouds
3,3,Tupaciguara,BR,-18.5922,-48.7050,76.37,70,72,3.15,moderate rain
4,4,Chesma,RU,53.8111,60.6533,-12.17,100,87,3.96,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Tupaciguara,BR,-18.5922,-48.7050,76.37,70,72,3.15,moderate rain
7,7,Kavaratti,IN,10.5669,72.6420,82.53,73,20,6.42,few clouds
8,8,Tres Arroyos,AR,-38.3739,-60.2798,80.20,30,7,8.97,clear sky
9,9,Hilo,US,19.7297,-155.0900,79.25,61,20,5.01,few clouds
13,13,Bengkulu,ID,-3.8004,102.2655,76.98,91,100,5.17,light rain
16,16,Hithadhoo,MV,-0.6000,73.0833,83.03,71,95,5.53,overcast clouds
17,17,Kismayo,SO,-0.3582,42.5454,78.64,80,83,15.64,broken clouds
19,19,Concordia,AR,-31.3930,-58.0209,76.78,51,2,8.01,clear sky
24,24,Marana,US,32.4367,-111.2254,77.58,9,0,0.00,clear sky
27,27,Rikitea,PF,-23.1203,-134.9692,80.08,71,100,15.08,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                214
City                   214
Country                214
Lat                    214
Lng                    214
Max Temp               214
Humidity               214
Cloudiness             214
Wind Speed             214
Current Description    214
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
nan_value = "NaN"
preferred_cities_df.replace("", nan_value, inplace=True)
preferred_cities_df.dropna(subset = ["Country"], inplace=True)
preferred_cities_df.count()

C:\Users\david\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:5244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
C:\Users\david\anaconda3\envs\PythonData\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


City_ID                214
City                   214
Country                214
Lat                    214
Lng                    214
Max Temp               214
Humidity               214
Cloudiness             214
Wind Speed             214
Current Description    214
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
clean_df = preferred_cities_df
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Tupaciguara,BR,76.37,moderate rain,-18.5922,-48.7050,
7,Kavaratti,IN,82.53,few clouds,10.5669,72.6420,
8,Tres Arroyos,AR,80.20,clear sky,-38.3739,-60.2798,
9,Hilo,US,79.25,few clouds,19.7297,-155.0900,
13,Bengkulu,ID,76.98,light rain,-3.8004,102.2655,
16,Hithadhoo,MV,83.03,overcast clouds,-0.6000,73.0833,
17,Kismayo,SO,78.64,broken clouds,-0.3582,42.5454,
19,Concordia,AR,76.78,clear sky,-31.3930,-58.0209,
24,Marana,US,77.58,clear sky,32.4367,-111.2254,
27,Rikitea,PF,80.08,overcast clouds,-23.1203,-134.9692,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!="",:]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Tupaciguara,BR,76.37,moderate rain,-18.5922,-48.7050,Hotel Tupaciguara
7,Kavaratti,IN,82.53,few clouds,10.5669,72.6420,Hotels in Lakshadweep Islands
8,Tres Arroyos,AR,80.20,clear sky,-38.3739,-60.2798,Habitacion Triple con baño y entrada privada
9,Hilo,US,79.25,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
13,Bengkulu,ID,76.98,light rain,-3.8004,102.2655,Grage Hotel Bengkulu
16,Hithadhoo,MV,83.03,overcast clouds,-0.6000,73.0833,Scoop Guest House
17,Kismayo,SO,78.64,broken clouds,-0.3582,42.5454,Agan-Hotel
19,Concordia,AR,76.78,clear sky,-31.3930,-58.0209,Palmar Hotel Casino
24,Marana,US,77.58,clear sky,32.4367,-111.2254,Valley of the Sun Mobile Home & RV Park 55+
27,Rikitea,PF,80.08,overcast clouds,-23.1203,-134.9692,People ThankYou


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "C:\\Users\\david\\Laura_Documents\\Rice_Data_Analytics_Course\\World_Weather_Analysis\\Vacation_Search\\WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))